In [ ]:
from tqdm.auto import tqdm
import re
import string

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
corpus = []
vectoriser = TfidfVectorizer()

In [ ]:
!hf auth login

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("Exploration-Lab/IL-TUR", "bail")

In [ ]:
# import json
# import pandas as pd
# def put_in_corpus(file):
#   global corpus
#   print(f"Current file:{file}")
#   with open(f"/scratch/username/train_test_data_for_modelling/{file}") as f:
#       data = json.load(f)
#   df = pd.DataFrame(data)
#   for i, row in tqdm(df.iterrows(),total=len(df)):
#       src_sents=[]
#       paras = df.loc[i]["segments"]['facts-and-arguments']
#       for para in paras:
#         sent = para.split('।')
#         sent = [i for i in sent if len(i)!=0 and i!=' ']
#         src_sents.extend(sent)
#       src_sents = list(filter(None, src_sents))
#       src_sents1=[]
#       for sent in src_sents:
#         try:
#           sent = ''.join([i for i in sent if not i.isdigit()])
#         except:
#           print(sent)
#         src_sents1.append(sent)
#       src_sents =src_sents1
#       corpus.extend(src_sents)

from tqdm import tqdm

def put_in_corpus_from_ds(split_name):
    global corpus
    dataset = ds[split_name]

    print(f"Processing split: {split_name}")

    for example in tqdm(dataset, total=len(dataset)):
        src_sents = []

        # text is a single long string
        text = str(example["text"]['facts-and-arguments'])

        # split into sentences (Hindi danda)
        sents = text.split('।')
        sents = [s for s in sents if s.strip()]

        # remove digits
        cleaned = []
        for sent in sents:
            sent = ''.join([c for c in sent if not c.isdigit()])
            cleaned.append(sent)

        corpus.extend(cleaned)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os

save_dir = "/content/drive/MyDrive/summaries_octtfidf2"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
put_in_corpus_from_ds('train_all')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
def tok(s):
  return s.split()
cv = CountVectorizer(tokenizer=tok)
data2 = cv.fit_transform(corpus)
tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(data2)
word2tfidf = dict(zip(cv.get_feature_names_out(), tfidf_transformer.idf_))

In [ ]:
def get_score(sentence):
  words = sentence.split()
  score = 0
  for word in words:
    try:
      score+= word2tfidf[word]
    except:
      pass
  return score

In [ ]:
# def process(file):
#     print(f"Current file:{file}")
#     with open(f"/scratch/username/train_test_data_for_modelling/{file}") as f:
#         data = json.load(f)
#     df = pd.DataFrame(data)
#     ranked_sentences = []
#     for i, row in tqdm(df.iterrows(),total=len(df)):
#         src_sents = df.loc[i]["segments"]['facts-and-arguments']
#         src_sents = [i.split('।') for i in src_sents]
#         # split all paragraphs in individual sentences
#         src_sents = [i for subl in src_sents for i in subl]
#         src_sents = [i for i in src_sents if len(i)!=0 and i!=' ']
#         src_sents = list(filter(None, src_sents))
#         src_sents1=[]
#         for sent in src_sents:
#           try:
#             sent = ''.join([i for i in sent if not i.isdigit()])
#           except:
#             print(sent)
#           src_sents1.append(sent)
#         src_sents =src_sents1
#         sentences = src_sents
#         scores=[]
#         for sent in sentences:
#           scores.append(get_score(sent))
#         ranks = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
#         ranks = [i[1] for i in ranks]
#         ranked_sentences.append(ranks)
#     df = df.head(len(ranked_sentences))
#     df['ranked-sentences'] = [i[:10] for i in ranked_sentences]
#     df['ranked-sentences'].map(len)
#     file = file.replace(".json",".csv")
#     df.to_csv(f"/scratch/username/summaries_octtfidf2/{file}")

from tqdm import tqdm
import pandas as pd
import re

def process_ds(split_name):
    print(f"Processing split: {split_name}")

    dataset = ds[split_name]
    ranked_sentences = []

    for example in tqdm(dataset, total=len(dataset)):
        # 1. get full text
        text = str(example["text"]['facts-and-arguments'])

        # 2. sentence split (Hindi danda)
        src_sents = text.split('।')

        # 3. clean sentences
        src_sents = [s.strip() for s in src_sents if s.strip()]

        cleaned = []
        for sent in src_sents:
            sent = re.sub(r'\d+', '', sent)   # remove digits
            cleaned.append(sent)

        sentences = cleaned

        # 4. scoring
        scores = []
        for sent in sentences:
            scores.append(get_score(sent))

        # 5. ranking
        ranks = sorted(
            ((scores[i], s) for i, s in enumerate(sentences)),
            reverse=True
        )
        ranks = [r[1] for r in ranks]

        ranked_sentences.append(ranks)

    # 6. convert to DataFrame for saving (to keep CSV output same)
    df = pd.DataFrame(dataset)

    df = df.head(len(ranked_sentences))
    df["ranked-sentences"] = [r[:10] for r in ranked_sentences]

    out_file = f"{split_name}_ranked.csv"
    df.to_csv(f"{save_dir}/{out_file}", index=False)

In [ ]:
splits = ["train_all","dev_all","test_all"]
for split in splits:
  process_ds(split)